In [1]:
import numpy as np
import matplotlib.pyplot as plt
def fonction_g_discrete(choix_angle,v):
    if v==0:
        print("ici")
        x=0
        y=0
    else: 
        if choix_angle==1:
            x=1
            y=0
        if choix_angle==2:
            x=-1
            y=0
        if choix_angle==3:
            x=0
            y=1
        if choix_angle==4:
            x=0
            y=-1
    return([x,y])



In [2]:
import random
import numpy as np
import pandas as pd

def overall():
    xcible=5
    ycible=5
    lim_x=[0,19]
    lim_y=[0,19]
    destination=[xcible,ycible]
    Liste_realisations=[]
    table_cout=np.random.multivariate_normal(size=20,mean=[-5]*20,cov=np.eye(20))
    goal_x=5
    goal_y=5
    for J in range(100):
        T=np.random.choice([20,50,100])
        Table_nom=[J]*T
        Recompenses=[]
        Actions=[]
        Nouvel_etat=[]
        Positions_t=[]
        x_si=np.random.choice(range(20))
        y_si=np.random.choice(range(20))
        situation_initiale=(x_si,y_si)
        for j in range(T):
            vitesse=np.random.choice([0,1,2,3])
            if situation_initiale[0]==goal_x and situation_initiale[1]==goal_y:
                recompense=1
            else:
                recompense=-1
            if vitesse!=0:
                choix_angle=random.choice([1,2,3,4])
                changement_pos=fonction_g_discrete(choix_angle=choix_angle,v=vitesse)
                situation_nouvelle=np.add(changement_pos,np.array(situation_initiale))
                if situation_nouvelle[0]<lim_x[0] or situation_nouvelle[0]>lim_x[1]:
                    recompense=+-20*vitesse
                    situation_nouvelle=situation_initiale
                elif situation_nouvelle[1]<lim_y[0] or situation_nouvelle[1]>lim_y[1]:
                    recompense=+-20*vitesse
                    situation_nouvelle=situation_initiale
                elif situation_nouvelle[0]==goal_x and situation_nouvelle[1]==goal_y:
                    recompense=1
                else:
                    recompense=+table_cout[situation_nouvelle[0],situation_nouvelle[1]]/vitesse
            else:
                situation_nouvelle=situation_initiale
                choix_angle=random.choice([0,1,2,3,4])
            Recompenses.append(recompense)
            Actions.append((vitesse,choix_angle))
            Positions_t.append(tuple(situation_initiale))
            Nouvel_etat.append(tuple(situation_nouvelle))
            situation_initiale=tuple(situation_nouvelle)
        dico=pd.DataFrame({"recompense":Recompenses,"actions":Actions,"etat_t":Positions_t,"etat_suivant":Nouvel_etat,"Indice_simul":Table_nom})
        Liste_realisations.append(dico)
    return(pd.concat(Liste_realisations))

In [3]:
R=overall()

In [4]:
R

,recompense,actions,etat_t,etat_suivant,Indice_simul
0,-4.156459,"(1, 4)","(8, 12)","(8, 11)",0
1,-1.449460,"(3, 2)","(8, 11)","(7, 11)",0
2,-4.431896,"(1, 3)","(7, 11)","(7, 12)",0
3,-4.348379,"(1, 4)","(7, 12)","(7, 11)",0
4,-5.446986,"(1, 2)","(7, 11)","(6, 11)",0
...,...,...,...,...,...
45,-3.068008,"(2, 4)","(9, 6)","(9, 5)",99
46,-2.574134,"(2, 4)","(9, 5)","(9, 4)",99
47,-6.136016,"(1, 3)","(9, 4)","(9, 5)",99
48,-5.270837,"(1, 3)","(9, 5)","(9, 6)",99


In [5]:
def q_learning(situations, eta, gamma):
    # Initialisation du dictionnaire Q avec des valeurs nulles
    liste_simulations = np.unique(situations["id_simulation"].tolist())
    Q={}
    for simulation in liste_simulations:
            sous_base = situations[situations["id_simulation"] == simulation]
            for row in sous_base.itertuples():
                etat_k = row.s_t
                action_k = row.a_t
                cle = (etat_k, action_k)
                Q[cle]=0
    
    
    threshold = 1e-3

    convergence = False
    while not convergence:
        Q_ant = dict(Q)  # Sauvegarde des valeurs de Q de l'itération précédente
        
        for simulation in liste_simulations:
            sous_base = situations[situations["id_simulation"] == simulation]
            for row in sous_base.itertuples():
                etat_k =row.s_t
                action_k = row.a_t
                cle = (etat_k, action_k)
                recompense = row.r_t
                situation_t_plus = row.s_t_p_1


                list_cle = [key for key in list(Q_ant.keys()) if key[0] == situation_t_plus]

                if len(list_cle) == 0:
                     Q_max = 0
                else:
                    Q_max = np.max([Q_ant[cle] for cle in list_cle])

                # Calcul de la valeur de Q en utilisant la formule de mise à jour de Q-learning
                # Q_max = np.max([Q_ant.get((situation_t_plus, int(a))) for a in actions["id"].tolist()])
                Q[cle] = (1 - eta) * Q_ant[cle] + eta * (recompense + gamma * Q_max)
        
        # Vérification de la convergence
        dist = sum(np.absolute(np.subtract(list(Q.values()),list(Q_ant.values()))))
        # convergence = all(abs(Q[k] - Q_ant[k]) < 1e-6 for k in Q.keys())
        if dist < threshold:
            convergence = True
    
    return Q



In [6]:
eta=0.7
gamma=0.6

In [7]:
R

,recompense,actions,etat_t,etat_suivant,Indice_simul
0,-4.156459,"(1, 4)","(8, 12)","(8, 11)",0
1,-1.449460,"(3, 2)","(8, 11)","(7, 11)",0
2,-4.431896,"(1, 3)","(7, 11)","(7, 12)",0
3,-4.348379,"(1, 4)","(7, 12)","(7, 11)",0
4,-5.446986,"(1, 2)","(7, 11)","(6, 11)",0
...,...,...,...,...,...
45,-3.068008,"(2, 4)","(9, 6)","(9, 5)",99
46,-2.574134,"(2, 4)","(9, 5)","(9, 4)",99
47,-6.136016,"(1, 3)","(9, 4)","(9, 5)",99
48,-5.270837,"(1, 3)","(9, 5)","(9, 6)",99


In [8]:
R = R.rename(columns={'recompense': 'r_t', 'actions': 'a_t',"Indice_simul":"id_simulation","etat_t":"s_t","etat_suivant":"s_t_p_1"})
R

,r_t,a_t,s_t,s_t_p_1,id_simulation
0,-4.156459,"(1, 4)","(8, 12)","(8, 11)",0
1,-1.449460,"(3, 2)","(8, 11)","(7, 11)",0
2,-4.431896,"(1, 3)","(7, 11)","(7, 12)",0
3,-4.348379,"(1, 4)","(7, 12)","(7, 11)",0
4,-5.446986,"(1, 2)","(7, 11)","(6, 11)",0
...,...,...,...,...,...
45,-3.068008,"(2, 4)","(9, 6)","(9, 5)",99
46,-2.574134,"(2, 4)","(9, 5)","(9, 4)",99
47,-6.136016,"(1, 3)","(9, 4)","(9, 5)",99
48,-5.270837,"(1, 3)","(9, 5)","(9, 6)",99


In [9]:
QL = q_learning(eta=eta,gamma=gamma,situations=R)

In [13]:
QL

{((8, 12), (1, 4)): -5.656458720366343,
 ((8, 11), (3, 2)): -3.983554621668626,
 ((7, 11), (1, 3)): -5.9318948328846774,
 ((7, 12), (1, 4)): -6.882473626498162,
 ((7, 11), (1, 2)): -6.946985327503194,
 ((6, 11), (1, 3)): -4.918099061013712,
 ((6, 12), (3, 4)): -3.3156613196466833,
 ((6, 11), (0, 2)): -2.4999993157184277,
 ((6, 11), (2, 2)): -3.6300093885414233,
 ((5, 11), (2, 2)): -3.706848599220804,
 ((4, 11), (3, 3)): -3.9339637930278104,
 ((4, 12), (1, 2)): -7.052769990191627,
 ((3, 12), (0, 0)): -2.4999993157184277,
 ((3, 12), (3, 2)): -2.792407028395665,
 ((2, 12), (0, 1)): -2.4999993157184277,
 ((2, 12), (3, 4)): -3.1376425951178373,
 ((2, 11), (1, 3)): -5.3772224537501385,
 ((2, 12), (1, 1)): -7.052769990191627,
 ((3, 12), (0, 3)): -2.4999993157184277,
 ((3, 12), (0, 2)): -2.4999993157184277,
 ((3, 12), (1, 2)): -5.3772224537501385,
 ((2, 12), (0, 0)): -2.4999993157184277,
 ((2, 12), (2, 1)): -4.276384652955027,
 ((3, 12), (0, 4)): -2.4999993157184277,
 ((3, 12), (3, 4)): -3.618

In [14]:
Etats=[]
best_action=[]
best_reward=[]
for cle in QL.keys():
    Liste_actions=[]
    Liste_recompense=[]
    Etats.append(cle[0])
    for cle2,valeur_recompense in QL.items():
        if cle[0]==cle2[0]:
            Liste_actions.append(cle2[1])
            Liste_recompense.append(valeur_recompense)
    indice_meilleur=np.argmax(Liste_recompense)
    MR_action=Liste_actions[indice_meilleur]
    MR_recompense=Liste_recompense[indice_meilleur]
    best_action.append(MR_action)
    best_reward.append(MR_recompense)
Best_action_per_state=pd.DataFrame({"etats":Etats,"Meilleure_action":best_action,"Meilleure_recompense":best_reward})

In [15]:
Best_action_per_state

,etats,Meilleure_action,Meilleure_recompense
0,"(8, 12)","(0, 3)",-2.499999
1,"(8, 11)","(0, 3)",-2.499999
2,"(7, 11)","(2, 2)",-4.223492
3,"(7, 12)","(0, 2)",-2.499999
4,"(7, 11)","(2, 2)",-4.223492
...,...,...,...
3417,"(10, 4)","(0, 2)",-2.499999
3418,"(10, 5)","(0, 2)",-2.499999
3419,"(9, 6)","(0, 2)",-2.499999
3420,"(9, 4)","(0, 2)",-2.499999
